# Дообучение модели Gemma-2-2b на всех данных

## Содержание ноутбука

Модель `Gemma-2-2b` дообучена на всех данных. Обучение производилось на GPU `NVIDIA A100 80Gb` в течении 7.5 часов.

**Гиперпараметры:**

* `LoRA rank = 16`
* `max_length = 256`
* `n_epochs = 2`
* `batch_size = 8`
* `learning_rate = 0.0005`
* `lr_scheduler = cosine`
* `weight_decay = 0.01`

**Результаты:**

* `loss = 1.9164`
* `eval_loss = 1.9087`

**Выводы:**

* Лучшие результаты получены в конце второй эпохи. Модель немного недообучена.

Примеры инференса в конце ноутбука.

## Логи MLFlow

![alt text](<../resources/mlflow/Screenshot 2024-12-17 032618.png>)

![alt text](<../resources/mlflow/Screenshot 2024-12-17 032424.png>)

## GPU информация

In [23]:
# !nvidia-smi

Mon Dec 16 21:32:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.03             Driver Version: 535.216.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:8C:00.0 Off |                    0 |
| N/A   35C    P0              84W / 500W |  22921MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## Импорты

In [1]:
# %pip -q install torch transformers datasets peft accelerate protobuf sentencepiece mlflow


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import random
import sys
from datetime import datetime

import mlflow
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from dotenv import load_dotenv
from huggingface_hub import login
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback, Trainer, TrainingArguments)

## Настройки

In [2]:
def seed_all(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

# Для повторяемости результатов
SEED = 42
seed_all(SEED)
# Загрузка переменных из .env файла
load_dotenv()
# Чтение токена
token = os.environ['HUGGING_FACE_ACCESS_TOKEN']
# Авторизация в Hugging Face
login(token)
# Устройство для тензорных вычислений и хранения модели в памяти.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Загрузка модели

In [7]:
model_name = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Половинная точность
    device_map="auto",
    token=token,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]


## Дообучение LoRA

In [8]:
LORA_RANK = 16  # Ранг матриц LoRA

# Настройки LoRA
lora_config = LoraConfig(
    r=LORA_RANK,  # Ранг малых матриц, веса которых мы будем обучать
    lora_alpha=LORA_RANK,  # (alpha / r) - множитель для выходов малых обучаемых матриц
    lora_dropout=0.1,  # Dropout регуляризация для малых обучаемых матриц
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

## Загрузка и очистка данных

In [7]:
dataset = load_dataset("nymless/geo-reviews-dataset-2023-prepared")

df = pd.DataFrame(dataset['train'])
df["name_ru"] = df["name_ru"].fillna("")
df["text"] = df["text"].str.replace("\\n", " ")
df = df.drop_duplicates(ignore_index=True)

dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['address', 'name_ru', 'rating', 'rubrics', 'text'],
    num_rows: 499799
})

## Векторизация данных

In [8]:
MAX_LENGTH = 256


def tokenize_text(examples):
    inputs = [
        f"Аddress: {address}\nName: {name}\nRating: {rating}\nKeywords: {rubrics}\nReview: {text}{tokenizer.eos_token}"
        for address, name, rating, rubrics, text in zip(
            examples["address"],
            examples["name_ru"],
            examples["rating"],
            examples["rubrics"],
            examples["text"],
        )
    ]
    # Векторизуем входной текст и приводим его к выбранной длине
    return tokenizer(
        inputs, max_length=MAX_LENGTH, padding="max_length", truncation=True
    )


dataset = dataset.map(tokenize_text, batched=True, remove_columns=dataset.column_names)
dataset

Map:   0%|          | 0/499799 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 499799
})

## Разделение данных

In [9]:
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=SEED)
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 449819
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 49980
    })
})

## Логирование MLFlow

In [12]:
MLFLOW_EXPERIMENT = "Gemma-2-2b-LoRA-Finetuning"

# Инициализация MLflow
mlflow.set_tracking_uri("./mlruns")  # Директория для логов в корне проекта
mlflow.set_experiment(MLFLOW_EXPERIMENT)  # Название эксперимента MLFlow

2024/12/16 13:31:47 INFO mlflow.tracking.fluent: Experiment with name 'Gemma-2-2b-LoRA-Finetuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/jupyter/work/resources/mlruns/702168699606335704', creation_time=1734355907824, experiment_id='702168699606335704', last_update_time=1734355907824, lifecycle_stage='active', name='Gemma-2-2b-LoRA-Finetuning', tags={}>

## Гиперпараметры обучения

In [13]:
BATCH_SIZE = 8
GRAD_ACCUM = 1
N_EPOCHS = 2
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 0.01

n_steps = int(round(N_EPOCHS * len(dataset["train"]) / BATCH_SIZE / GRAD_ACCUM, 0))
print("Всего шагов обучения:", n_steps)

Всего шагов обучения: 112455


In [14]:
HUB_MODEL_ID = "nymless/gemma-2-2b-lora-finetuned"

training_args = TrainingArguments(
    # Включаем логирование в MLFlow
    report_to="mlflow",
    logging_steps=500,  # Через сколько шагов логировать
    # Генерация имени MLFlow Run с текущим временем и датой
    run_name=f"{MLFLOW_EXPERIMENT}-{datetime.now().strftime('%d.%m.%Y, %H:%M:%S')}",
    # Директория сохранения модели
    output_dir="./models/gemma-2-2b-lora-finetuned",
    # Параметры оценки и сохранения модели
    overwrite_output_dir=True,  # Перезапись директории при каждом запуске обучения
    save_strategy="epoch",  # Сохраняем модель в конце каждой эпохи
    load_best_model_at_end=True,  # Сохраняем лучшую по метрике модель в конце обучения
    metric_for_best_model="loss",  # Метрика для оценки модели
    save_total_limit=1,  # Сохранить только одну модель
    eval_strategy="epoch",  # Оценивать модель в конце каждой эпохи
    # Гиперпараметры
    learning_rate=LEARNING_RATE,  # Скорость обучения
    lr_scheduler_type="cosine",
    per_device_train_batch_size=BATCH_SIZE,  # Размер батча данных на этапе обучения
    per_device_eval_batch_size=BATCH_SIZE,  # Размер батча данных на этапе оценивания
    gradient_accumulation_steps=GRAD_ACCUM,  # Накапливать градиенты N шагов и подстроить веса
    num_train_epochs=N_EPOCHS,  # Количество эпох
    weight_decay=WEIGHT_DECAY,  # Коэффициент регуляризации
    fp16=True,  # Использовать формат чисел FP16 (Floating point 16-bit)
    # Работа с Hugging Face Hub (Отключено)
    push_to_hub=False,  # Сохранение модели в Hub со стандартным commit_message
    hub_model_id=HUB_MODEL_ID,  # Название модели в Hub
    hub_strategy="end",  # Загружать модель в Hub в конце обучения
    hub_token=token,  # Токен Hugging Face
)

In [15]:
# Объект, который извлекает данные батчами для обучения
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Отключаем режим маскирования текста MLM (Masked language modeling)
)

In [16]:
# Объект, который управляет ранней остановкой обучения
early_topping_callback = EarlyStoppingCallback(
    early_stopping_patience=2, early_stopping_threshold=0.001
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    callbacks=[early_topping_callback]
)

## Обучение модели

In [18]:
trainer.train(
    resume_from_checkpoint=False, # Продолжить обучение с выбранного (или последнего) чекпоинта
)

  0%|          | 500/112456 [02:02<7:35:25,  4.10it/s]

{'loss': 2.1569, 'grad_norm': 0.518764853477478, 'learning_rate': 0.0004999756119294758, 'epoch': 0.01}


  1%|          | 1000/112456 [04:04<7:34:04,  4.09it/s]

{'loss': 2.1077, 'grad_norm': 0.6214762330055237, 'learning_rate': 0.0004999024524761271, 'epoch': 0.02}


  1%|▏         | 1500/112456 [06:06<7:31:32,  4.10it/s]

...

{'eval_loss': 1.9087095260620117, 'eval_runtime': 669.5652, 'eval_samples_per_second': 74.645, 'eval_steps_per_second': 9.331, 'epoch': 2.0}


/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: debb7e44-ac6a-4572-8727-22a70f75fd82)') - silently ignoring the lookup for the file config.json in google/gemma-2-2b.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/gemma-2-2b - will assume that the vocabulary was not modified.
  warnings.warn(
100%|██████████| 112456/112456 [7:58:47<00:00,  4.56it/s]

{'train_runtime': 28727.5111, 'train_samples_per_second': 31.316, 'train_steps_per_second': 3.915, 'train_loss': 1.967983855083113, 'epoch': 2.0}


100%|██████████| 112456/112456 [7:58:48<00:00,  3.91it/s]


TrainOutput(global_step=112456, training_loss=1.967983855083113, metrics={'train_runtime': 28727.5111, 'train_samples_per_second': 31.316, 'train_steps_per_second': 3.915, 'total_flos': 2.8019826572973834e+18, 'train_loss': 1.967983855083113, 'epoch': 2.0})

## Сохранение модели

<https://huggingface.co/nymless/gemma-2-2b-lora-finetuned>

In [19]:
trainer.push_to_hub("Train on all data")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]
training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]


adapter_model.safetensors:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 123k/34.4M [00:00<00:36, 933kB/s]


adapter_model.safetensors:   1%|          | 123k/12.8M [00:00<00:13, 917kB/s]

tokenizer.json:   2%|▏         | 623k/34.4M [00:00<00:12, 2.76MB/s]


training_args.bin: 100%|██████████| 5.50k/5.50k [00:00<00:00, 20.5kB/s]42MB/s]



adapter_model.safetensors:  29%|██▉       | 3.70M/12.8M [00:00<00:00, 12.5MB/s]

tokenizer.json:   8%|▊         | 2.61M/34.4M [00:00<00:03, 7.99MB/s]


adapter_model.safetensors:  71%|███████   | 9.04M/12.8M [00:00<00:00, 24.3MB/s]

tokenizer.json:  17%|█▋        | 5.88M/34.4M [00:00<00:02, 13.4MB/s]

adapter_model.safetensors: 100%|██████████| 12.8M/12.8M [00:00<00:00, 14.0MB/s]


Upload 3 LFS files:  33%|███▎      | 1/3 [00:01<00:02,  1.3

CommitInfo(commit_url='https://huggingface.co/nymless/gemma-2-2b-lora-finetuned/commit/45f2fa0012e30b64c9531f2f0d773c40801b6505', commit_message='Train on all data', commit_description='', oid='45f2fa0012e30b64c9531f2f0d773c40801b6505', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nymless/gemma-2-2b-lora-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='nymless/gemma-2-2b-lora-finetuned'), pr_revision=None, pr_num=None)

## Загрузка модели

In [ ]:
model_name = "google/gemma-2-2b"
lora_finetuned = "nymless/gemma-2-2b-lora-finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoPeftModelForCausalLM.from_pretrained(
    lora_finetuned,
    token=token,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Проверка генерации текста

In [10]:
# Возьмём векторизованный текст из тестового датасета
vectorized_text = dataset["test"][0]["input_ids"]
# Декодируем из вектора обратно в текст
text = tokenizer.decode(vectorized_text, skip_special_tokens=True)
print(text)

Аddress: Московская область, городской округ Балашиха, деревня Пестово, 1А
Name: Грант
Rating: 1
Keywords: Магазин автозапчастей и автотоваров;Автокосметика, автохимия;Аккумуляторы и зарядные устройства
Review: У сотрудников информация по запчасти разнятся, очень не понравилось отношение сотрудников 



In [5]:
sys.path.append(os.path.abspath(".."))

from validation.Inputs import Inputs
from scripts.generate import generate

inputs = Inputs(
    address="Московская область, городской округ Балашиха, деревня Пестово, 1А",
    name="Грант",
    rating="1",
    rubrics="Магазин автозапчастей и автотоваров;Автокосметика, автохимия;Аккумуляторы и зарядные устройства",
)
review = generate(model, tokenizer, inputs, max_length=512)
print("Review:", review)

Review: 08.03.2023. С 19.12.2022 года заказываю автоаксессуары по каталогу. 02.03.2023. Доставка отменяется на 25.03.2023. До 23.03.2023. Доставки не было. Заказ отменяется. Сроки доставки отменяются без объяснения причин.  
